In [ ]:
!pip install keras-tcn --quiet


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

# Load the dataset
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/dataset/Data_Historis_BBRI_Train.csv')

# Mengubah kolom 'Tanggal' menjadi datetime
df['Tanggal'] = pd.to_datetime(df['Tanggal'], format='%d/%m/%Y')
df = df.sort_values('Tanggal')
df = df.set_index('Tanggal')

df.head()

Mounted at /content/drive


,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
Tanggal,,,,,,
2019-01-02,3.61,3.61,3.64,3.59,"74,95M","-1,37%"
2019-01-03,3.62,3.58,3.64,3.58,"93,46M","0,28%"
2019-01-04,3.66,3.62,3.66,3.61,"114,46M","1,10%"
2019-01-07,3.66,3.70,3.71,3.66,"74,64M","0,00%"
2019-01-08,3.68,3.66,3.68,3.63,"81,36M","0,55%"


In [ ]:
def convert_to_float(x):
    x = str(x).strip().upper()         # Pastikan huruf besar
    x = x.replace(',', '.')            # Ganti koma → titik

    # Jika ada B
    if 'B' in x:
        num = float(x.replace('B', ''))
        return num * 1_000

    # Jika ada M
    elif 'M' in x:
        num = float(x.replace('M', ''))
        return num * 1

    # Jika angka biasa (misal "1200000" atau "1.200.000")
    else:
        # Hapus titik pemisah ribuan
        num = x.replace('.', '')
        return float(num)

def convert_to_float_persen(x):
    x = str(x).strip().upper()         # Pastikan huruf besar
    x = x.replace(',', '.')            # Ganti koma → titik

    # Jika ada B
    if '%' in x:
        num = float(x.replace('%', ''))
        return num * 1

    # Jika angka biasa (misal "1200000" atau "1.200.000")
    else:
        # Hapus titik pemisah ribuan
        num = x.replace('.', '')
        return float(num)


df['Vol.'] = df['Vol.'].apply(convert_to_float)
df['Perubahan%'] = df['Perubahan%'].apply(convert_to_float_persen)

df.head()

,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
Tanggal,,,,,,
2019-01-02,3.61,3.61,3.64,3.59,74.95,-1.37
2019-01-03,3.62,3.58,3.64,3.58,93.46,0.28
2019-01-04,3.66,3.62,3.66,3.61,114.46,1.10
2019-01-07,3.66,3.70,3.71,3.66,74.64,0.00
2019-01-08,3.68,3.66,3.68,3.63,81.36,0.55


In [ ]:
df.isnull().sum()

,0
Terakhir,0
Pembukaan,0
Tertinggi,0
Terendah,0
Vol.,0
Perubahan%,0


In [ ]:
df

,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
Tanggal,,,,,,
2019-01-02,3.610,3.61,3.640,3.590,74.95,-1.37
2019-01-03,3.620,3.58,3.640,3.580,93.46,0.28
2019-01-04,3.660,3.62,3.660,3.610,114.46,1.10
2019-01-07,3.660,3.70,3.710,3.660,74.64,0.00
2019-01-08,3.680,3.66,3.680,3.630,81.36,0.55
...,...,...,...,...,...,...
2023-12-21,5.575,5.55,5.600,5.525,99.05,0.45
2023-12-22,5.675,5.65,5.700,5.600,109.41,1.79
2023-12-27,5.625,5.70,5.725,5.625,131.37,-0.88


In [ ]:
# # Select features and target
# features = ['Terakhir', 'Pembukaan','Tertinggi','Terendah','Vol.','Perubahan%']
# target = 'Terakhir'

# Anggap 'df' adalah dataframe Anda
# 1. Buat fitur baru dengan menggeser data ke bawah (Lagging)
# Ini berarti 'Terakhir_Lag1' berisi harga 'Terakhir' dari hari sebelumnya.
df['Terakhir_Prev'] = df['Terakhir'].shift(1)
df['Pembukaan_Prev'] = df['Pembukaan'].shift(1)
df['Vol_Prev'] = df['Vol.'].shift(1)

# Hapus baris paling atas yang sekarang menjadi NaN (karena di-shift)
df = df.dropna()

# 2. Definisikan Features dan Target Baru
# Gunakan data 'Kemarin' untuk memprediksi 'Hari Ini'
features = ['Terakhir_Prev', 'Pembukaan_Prev', 'Vol_Prev', 'Tertinggi', 'Terendah']
# Catatan: Pastikan 'Tertinggi' dan 'Terendah' juga dari masa lalu jika Anda ingin memprediksi harga Close hari ini sebelum pasar tutup.

target = 'Terakhir' # Target tetap harga asli hari ini

X = df[features]
y = df[target]

print("Setup Fitur Berhasil.")
print(df.info())

Setup Fitur Berhasil.
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1218 entries, 2019-01-03 to 2023-12-29
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Terakhir        1218 non-null   float64
 1   Pembukaan       1218 non-null   float64
 2   Tertinggi       1218 non-null   float64
 3   Terendah        1218 non-null   float64
 4   Vol.            1218 non-null   float64
 5   Perubahan%      1218 non-null   float64
 6   Terakhir_Prev   1218 non-null   float64
 7   Pembukaan_Prev  1218 non-null   float64
 8   Vol_Prev        1218 non-null   float64
dtypes: float64(9)
memory usage: 95.2 KB
None


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])


In [ ]:
# Scale data
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_x.fit_transform(df[features])
y_scaled = scaler_y.fit_transform(df[[target]])


In [ ]:
scaler_x

MinMaxScaler()

In [ ]:
scaler_y

MinMaxScaler()

In [ ]:
scaled_data

array([[0.40506329, 0.3942029 , 0.05594546, 0.39367816, 0.40398293],
       [0.40787623, 0.38550725, 0.0778526 , 0.39942529, 0.41251778],
       [0.41912799, 0.39710145, 0.10270673, 0.4137931 , 0.42674253],
       ...,
       [0.9859353 , 0.98550725, 0.09672991, 0.99281609, 0.98577525],
       [0.9718706 , 1.        , 0.12272023, 1.        , 1.        ],
       [1.        , 1.        , 0.11095594, 1.        , 1.        ]])

In [ ]:
df.isnull().sum()
df

,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%,Terakhir_Prev,Pembukaan_Prev,Vol_Prev
Tanggal,,,,,,,,,
2019-01-03,3.620,3.58,3.640,3.580,93.46,0.28,3.610,3.61,74.95
2019-01-04,3.660,3.62,3.660,3.610,114.46,1.10,3.620,3.58,93.46
2019-01-07,3.660,3.70,3.710,3.660,74.64,0.00,3.660,3.62,114.46
2019-01-08,3.680,3.66,3.680,3.630,81.36,0.55,3.660,3.70,74.64
2019-01-09,3.720,3.71,3.740,3.690,119.39,1.09,3.680,3.66,81.36
...,...,...,...,...,...,...,...,...,...
2023-12-21,5.575,5.55,5.600,5.525,99.05,0.45,5.550,5.70,138.47
2023-12-22,5.675,5.65,5.700,5.600,109.41,1.79,5.575,5.55,99.05
2023-12-27,5.625,5.70,5.725,5.625,131.37,-0.88,5.675,5.65,109.41


In [ ]:
def create_multifeature_dataset(data, target, window=60):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i])
        y.append(target[i, 0])
    return np.array(X), np.array(y)

WINDOW = 15 # Changed WINDOW size to 30

X_seq, y_seq = create_multifeature_dataset(
    X_scaled,
    y_scaled,
    WINDOW
)

# Bentuk input → (batch, timestep, fitur)
print(X_seq.shape)

(1203, 15, 5)


In [ ]:
# Train-test split
split = int(len(X_seq)*0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

In [ ]:
# Build TCN model
model = Sequential([
    TCN(input_shape=(WINDOW, len(features))),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Train
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.1) # Increased epochs to 100

/usr/local/lib/python3.12/dist-packages/tcn/tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 16s 91ms/step - loss: 4.4706 - val_loss: 0.4082
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.2331 - val_loss: 0.1419
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 0.0416 - val_loss: 0.0149
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - loss: 0.0159 - val_loss: 0.0162
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0118 - val_loss: 0.0057
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0067 - val_loss: 0.0049
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - loss: 0.0054 - val_loss: 0.0041
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 179ms/step - loss: 0.0041 - val_loss: 0.0036
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0040 - val_loss: 0.0046
Epoch 10/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0040 - val_loss: 0.0026
Epoch 11/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0035 - val_loss: 0.0027
Epoch 12/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss:

In [ ]:
# Predict
pred_scaled = model.predict(X_test)
pred = scaler_y.inverse_transform(pred_scaled)
y_true = scaler_y.inverse_transform(y_test.reshape(-1, 1))

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step


In [ ]:
future_days = 7
predictions = []

# Get the last sequence of WINDOW data points from X_scaled
last_sequence = X_scaled[-WINDOW:].reshape(1, WINDOW, len(features))

for _ in range(future_days):
    # Predict the next value
    next_prediction_scaled = model.predict(last_sequence)[0, 0]
    predictions.append(next_prediction_scaled)

    # Update the sequence: remove the oldest and add the new prediction
    # For simplicity, we are assuming the features will follow the predicted pattern
    # In a real-world scenario, you might have external features for future dates

    # Create a dummy next feature vector (assuming target is the first feature to be updated)
    # This is a simplification; ideally, you would have a way to estimate future features
    # The other features are assumed to be the same as the last observed feature vector
    next_feature_vector = np.concatenate(([next_prediction_scaled], last_sequence[0, -1, 1:]))

    # Reshape next_feature_vector to (1, 1, num_features) to append
    next_feature_vector = next_feature_vector.reshape(1, 1, len(features))

    # Update last_sequence by dropping the first element and appending the new prediction
    last_sequence = np.concatenate((last_sequence[:, 1:, :], next_feature_vector), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [ ]:
# Inverse transform the predictions to the original scale
future_predictions = scaler_y.inverse_transform(np.array(predictions).reshape(-1, 1))

# Generate future dates
last_date = df.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days)

# Create a DataFrame for future predictions
future_df = pd.DataFrame({'Tanggal': future_dates, 'Prediksi_Terakhir': future_predictions.flatten()})
future_df.set_index('Tanggal', inplace=True)
display(future_df)

,Prediksi_Terakhir
Tanggal,
2023-12-30,5.663853
2023-12-31,5.719008
2024-01-01,5.744772
2024-01-02,5.609776
2024-01-03,5.701689
2024-01-04,5.700150
2024-01-05,5.568052


In [ ]:
mae = mean_absolute_error(y_true, pred)
rmse = np.sqrt(mean_squared_error(y_true, pred))

print("MAE:", mae)
print("RMSE:", rmse)

MAE: 0.08592860020047895
RMSE: 0.10608179990511979
